# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [3]:
earthquake_json['metadata']

{'generated': 1669958798000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2022-11-01&endtime=2022-12-01',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.13.6',
 'count': 10649}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 0.3,
  'place': '6km WNW of Cobb, CA',
  'time': 1669852609560,
  'updated': 1669858814459,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc73813271',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc73813271&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 1,
  'net': 'nc',
  'code': '73813271',
  'ids': ',nc73813271,',
  'sources': ',nc,',
  'types': ',nearby-cities,origin,phase-data,scitech-link,',
  'nst': 5,
  'dmin': 0.003242,
  'rms': 0.01,
  'gap': 119,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 0.3 - 6km WNW of Cobb, CA'},
 'geometry': {'type': 'Point', 'coordinates': [-122.7885, 38.8355, 2.18]},
 'id': 'nc73813271'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [8]:
earthquake_json['features'][0]['geometry']

{'type': 'Point', 'coordinates': [-122.7885, 38.8355, 2.18]}

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,0.30,"6km WNW of Cobb, CA",1669852609560,1669858814459,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73813271,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",5.0,0.003242,0.01,119.0,md,earthquake,"M 0.3 - 6km WNW of Cobb, CA"
1,0.80,"20 km WNW of Tatitlek, Alaska",1669852590980,1669859619642,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak022fcriwxh,",",ak,",",origin,phase-data,",NaN,NaN,0.46,NaN,ml,earthquake,"M 0.8 - 20 km WNW of Tatitlek, Alaska"
2,0.40,"87 km NNW of Karluk, Alaska",1669852053668,1669872363180,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",av91724361,ak022fcrh13q,",",av,ak,",",origin,phase-data,",NaN,NaN,0.25,NaN,ml,earthquake,"M 0.4 - 87 km NNW of Karluk, Alaska"
3,-0.50,Alaska Peninsula,1669851240520,1669872664740,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",av91724346,",",av,",",origin,phase-data,",4.0,NaN,0.06,158.0,ml,earthquake,M -0.5 - Alaska Peninsula
4,-0.17,"7km NW of The Geysers, CA",1669850830940,1669857074354,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73813266,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",14.0,0.003119,0.02,63.0,md,earthquake,"M -0.2 - 7km NW of The Geysers, CA"


### (Optional) Write Data to CSV

In [9]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>